In [ ]:
import datetime
import json
import random
import math
import re
from decimal import Decimal
from multiprocessing import Pool
from random import randint

import numpy as np
import pandas as pd
from pymongo import MongoClient, ASCENDING, DESCENDING

from core.config import settings, client, web3
from policy.uniswap_v3.offchain import OffchainUniswapV3
from policy.uniswap_v3.analytics import process_dynamic_curve_AMM_slippage
from policy.uniswap_v3.utils import getTickAtSqrtRatio
from core.subgraph.downloader import DataDownloader
from policy.uniswap_v3.subgraph import LiquiditySnapshot

# Chart
from bokeh.models import BasicTickFormatter
from bokeh.io import output_notebook
from bokeh.plotting import figure, show, save
from bokeh.resources import CDN
from bokeh.embed import file_html

output_notebook()

## Contract Addresses to run

In [ ]:
# MongoQuery: Find record based on key value pair; and project only some keys; and dispol

all_registered = list(client["UniswapV3Jun082022"]["pool_data"].find({"registered":True},{"last_updated_swap":1, "_id":0, "pool":1}))
shortlisted_pools = pd.DataFrame(all_registered).sort_values("last_updated_swap", ascending=False)
shortlisted_pools

In [ ]:
market = "0xC2e9F25Be6257c210d7Adf0D4Cd6E3E881ba25f8".lower()
market

## Processing 

In [ ]:
# MongoQuery: Find a pool by contract address from pool_data collection

list(client[settings.DB_ORACLE][settings.COLL_POOL_DATA].find({"pool":"0xC2e9F25Be6257c210d7Adf0D4Cd6E3E881ba25f8"}))

In [ ]:
# MongoQuery: Find 'end_date' value querying by pool address; project some fields; sort by date; and limit to only 1 record

processed_swaps = list(client[settings.DB_ANALYTICS][settings.COLL_PRICE_DISTRIBUTIONS].find({"contract_address":market},{"pool":1, "contract_address":1, "to_date":1, "from_date":1}).sort("to_date",ASCENDING).limit(1))
if not processed_swaps:
    print("Gather from uniswap pool as the data has never been processed")
    
    to_date_from_uniswap = list(client[settings.DB_ORACLE][settings.COLL_POOL_DATA].find({"pool":web3.toChecksumAddress(market)}))

to_date_from_uniswap[0].get("evt_block_time")

In [ ]:
datetime.datetime.fromisoformat(to_date_from_uniswap[0].get("evt_block_time")).date() + datetime.timedelta(days=1) #.isoformat()

In [ ]:
historical_days = 7

processed_swaps = list(client[settings.DB_ANALYTICS][settings.COLL_PRICE_DISTRIBUTIONS].find({"contract_address":market},{"pool":1, "contract_address":1, "to_date":1, "from_date":1}).sort("to_date",ASCENDING).limit(1))
if not processed_swaps:
    print("Gather from uniswap pool as the data has never been processed")
    to_date_from_uniswap = list(client[settings.DB_ORACLE][settings.COLL_POOL_DATA].find({"pool":web3.toChecksumAddress(market)}))
    
    start_date = datetime.datetime.fromisoformat(to_date_from_uniswap[0].get("evt_block_time")).date().isoformat()
    end_date = (datetime.datetime.fromisoformat(to_date_from_uniswap[0].get("evt_block_time")).date() + datetime.timedelta(days=historical_days+1)).isoformat()
else:
    start_date = datetime.datetime.fromisoformat(processed_swaps[0].get("evt_block_time")).date().isoformat()
    end_date = (datetime.datetime.fromisoformat(processed_swaps[0].get("evt_block_time")).date() + datetime.timedelta(days=historical_days+1)).isoformat()


# end_date = datetime.datetime.now().isoformat()
# start_date = (datetime.datetime.now() - datetime.timedelta(days=historical_days)).isoformat()
colors = ["#2596be", "#0b64c5", "#Ff00f6", "#700639", "#873e23", "#637006", "#70066e", "purple"]

uniswap_simulator = OffchainUniswapV3(contract_address=market)
token_name = uniswap_simulator.token0_name + "-" + uniswap_simulator.token1_name + " " + str(
    round(uniswap_simulator.fee * (10 ** 6)) / (10 ** 4)) + "%"

title = f"Distributions of different prices" + " | " + start_date.split("T")[0] + " To " + end_date.split("T")[
    0] + " | " + "UniswapV3 Pool - " + token_name


# run once before for loop
# uniswap_simulator.gather_mint_transactions()
# uniswap_simulator.gather_burn_transactions()


f"Last synchronised with UniswapV3: {uniswap_simulator.latest_pool_time}"

#### Download swap transactions data (either from mongoDB or subgraph)

In [ ]:
all_swap_transactions_raw: pd.DataFrame = DataDownloader.get_historical_swaps(market.lower(),
                                                                               datetime.timedelta(
                                                                                   hours=historical_days * 24))
    
# all_swap_transactions_raw.head()

# start_date, end_date

# LOG INDEX IS MISSING. Which means we dont' know the exact order of the swaps which kinda ruins the ;


# MongoQuery: Find a pool by contract address and by 'evt_block_time' between two timestamps.
all_swap_transactions_raw = pd.DataFrame(client[settings.DB_ORACLE][settings.COLL_ORACLE_SWAP].find({
   "contract_address":web3.toChecksumAddress(market),
  "evt_block_time": {"$gte":start_date, "$lte":end_date}}))
all_swap_transactions_raw

#### Data cleaning and processing

In [ ]:
all_swap_transactions_raw.timestamp = pd.to_datetime(all_swap_transactions_raw.timestamp, unit='s')
all_swap_transactions_raw.rename({
    'transaction.blockNumber': 'evt_block_number',
    'timestamp': 'evt_block_time',
    'id': 'evt_tx_hash'
}, axis=1, inplace=True)

# def remove_hash(row):
#     return row.evt_tx_hash.split("#")[0]

# all_swap_transactions_raw.evt_tx_hash = all_swap_transactions_raw.apply(remove_hash, axis=1)
all_swap_transactions_raw["logIndex"] = all_swap_transactions_raw.logIndex.astype(int)
all_swap_transactions_raw.sort_values(["evt_block_number", "logIndex"], ascending=[True, True], inplace=True)
all_swap_transactions_raw.drop_duplicates(subset="evt_tx_hash", keep='first', inplace=True)

all_swap_transactions = all_swap_transactions_raw[
    ['evt_block_time', "evt_block_number", "evt_tx_hash", "logIndex", "sqrtPriceX96", "amount0", "amount1",
     'tick']].copy()

all_swap_transactions.sort_values(["evt_block_number", "logIndex"], ascending=[True, True], inplace=True)

all_swap_transactions["previous_tick"] = all_swap_transactions.tick.shift(1)

all_swap_transactions["token0"] = uniswap_simulator.token0_name
all_swap_transactions["token1"] = uniswap_simulator.token1_name
all_swap_transactions["fee"] = uniswap_simulator.fee

def price_from_sqrtPriceX96_in_token1(row):
    return (int(row.sqrtPriceX96) ** 2) / (2 ** 192) * (
                10 ** -(uniswap_simulator.token1_decimals - uniswap_simulator.token0_decimals))

all_swap_transactions["after_swap_price_in_token1"] = all_swap_transactions.apply(price_from_sqrtPriceX96_in_token1,
                                                                                  axis=1)
all_swap_transactions["after_swap_price_in_token0"] = 1 / all_swap_transactions["after_swap_price_in_token1"]

all_swap_transactions["before_swap_price_in_token1"] = all_swap_transactions.after_swap_price_in_token1.shift(1)
all_swap_transactions["before_swap_price_in_token0"] = all_swap_transactions.after_swap_price_in_token0.shift(1)

all_swap_transactions = all_swap_transactions.iloc[1:]
all_swap_transactions["previous_tick"] = all_swap_transactions.previous_tick.astype("int")

def get_amount_swapped(row):

    amount0 = row.amount0  # * (10 ** -uniswap_simulator.token0_decimals)
    amount1 = row.amount1  # * (10 ** -uniswap_simulator.token1_decimals)

    if amount0 > 0:
        return amount0
    elif amount1 > 0:
        return amount1
    else:
        return

def get_amount_received(row):

    amount0 = row.amount0  # * (10 ** -uniswap_simulator.token0_decimals)
    amount1 = row.amount1  # * (10 ** -uniswap_simulator.token1_decimals)

    if amount0 < 0:
        return amount0
    elif amount1 < 0:
        return amount1
    else:
        return

all_swap_transactions["amount_received"] = all_swap_transactions.apply(get_amount_received, axis=1) * -1
all_swap_transactions["amount_swapped_gross"] = all_swap_transactions.apply(get_amount_swapped, axis=1)
all_swap_transactions["amount_swapped_fee"] = all_swap_transactions["amount_swapped_gross"] * all_swap_transactions[
    "fee"]
all_swap_transactions["amount_swapped_net"] = (all_swap_transactions["amount_swapped_gross"]) - all_swap_transactions.amount_swapped_fee

# Calculate amount received without slippage

def get_amount_received_without_slippage_gross(row):
    amount0 = row.amount0  # * (10 ** -uniswap_simulator.token0_decimals)
    amount1 = row.amount1  # * (10 ** -uniswap_simulator.token1_decimals)

    # using amount_swapped_net

    if amount0 > 0:
        return row.amount_swapped_gross / row.before_swap_price_in_token0
    elif amount1 > 0:
        return row.amount_swapped_gross / row.before_swap_price_in_token1
    else:
        return

all_swap_transactions["amount_received_without_slippage_gross"] = all_swap_transactions.apply(
    get_amount_received_without_slippage_gross, axis=1)

def get_amount_received_without_slippage_net(row):
    amount0 = row.amount0  # * (10 ** -uniswap_simulator.token0_decimals)
    amount1 = row.amount1  # * (10 ** -uniswap_simulator.token1_decimals)

    # using amount_swapped_net

    if amount0 > 0:
        return row.amount_swapped_net / row.before_swap_price_in_token0
    elif amount1 > 0:
        return row.amount_swapped_net / row.before_swap_price_in_token1
    else:
        return

all_swap_transactions["amount_received_without_slippage_net"] = all_swap_transactions.apply(
    get_amount_received_without_slippage_net, axis=1)

all_swap_transactions["slippage_gross"] = (all_swap_transactions['amount_received_without_slippage_gross'] -
                                           all_swap_transactions["amount_received"]) / all_swap_transactions[
                                              "amount_received"]
all_swap_transactions["slippage_net"] = (all_swap_transactions['amount_received_without_slippage_net'] -
                                         all_swap_transactions["amount_received"]) / all_swap_transactions[
                                            "amount_received"]

def get_average_trade_price_in_token1_net(row):
    amount0 = row.amount0  # * (10 ** -uniswap_simulator.token0_decimals)
    amount1 = row.amount1  # * (10 ** -uniswap_simulator.token1_decimals)

    # using amount_swapped_net

    if amount0 > 0:
        return row.amount_received / row.amount_swapped_net
    elif amount1 > 0:
        return row.amount_swapped_net / row.amount_received
    else:
        return

all_swap_transactions["average_trading_price_token1_net"] = all_swap_transactions.apply(get_average_trade_price_in_token1_net, axis=1)

def get_average_trade_price_in_token1_gross(row):
    amount0 = row.amount0  # * (10 ** -uniswap_simulator.token0_decimals)
    amount1 = row.amount1  # * (10 ** -uniswap_simulator.token1_decimals)

    # using amount_swapped_net

    if amount0 > 0:
        return row.amount_received / row.amount_swapped_gross
    elif amount1 > 0:
        return row.amount_swapped_gross / row.amount_received
    else:
        return

all_swap_transactions["average_trading_price_token1_gross"] = all_swap_transactions.apply(get_average_trade_price_in_token1_net, axis=1)

all_swap_transactions.set_index("evt_block_number", inplace=True)